# ADCS Simulator and System Modeling / Selection Notebook 
Follow the readme to install dependencies and correctly configure you environment/machine to run the varios blocks.

In [5]:
%matplotlib widget
import numpy as np
import math

In [6]:
import lib.physics as physics
import lib.cubesat as cubesat
import lib.renderer as renderer

In [7]:
def vcircular(altitude):
    return math.sqrt(physics.Consts.mu / (altitude + physics.Consts.R_earth))

initial_state  = cubesat.State(physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude), 0) # x, y, z, dx, dy, dz
qubesat = cubesat.Cubesat(1, length=0.1, width=0.1, height=0.1) # mass in kg, length, width, height in m

planet  = physics.Planet(physics.Consts.M_earth, physics.Consts.R_earth) # mass in kg, radius in meters
# planet  = physics.Planet(10, 200) # mass in kg, radius in meters

In [8]:
class Simulator:
    def __init__(self, time, cubesat, planet, dt=0.1):
        """
        time and step size in seconds
        """
        self.dt = dt
        self.time = time
        self.cubesat = cubesat
        self.planet = planet
        self.elapsed_time = 0
        
        # The new state is provided by the following equation
        # xk = Axk-1 + Buk-1 + wk-1
        # u is the vector sum of all the accelerations
        
        # 3D state matrix - A
        self.A = np.array([[1, 0, 0, self.dt, 0, 0],
                          [0, 1, 0, 0, self.dt, 0],
                          [0, 0, 1, 0, 0, self.dt],
                          [0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 0, 1]]) 
        
        # 3D control matrix - B
        self.B = np.array([[1/2*self.dt**2, 0, 0],
                          [0, 1/2*self.dt**2, 0],
                          [0, 0, 1/2*self.dt**2],
                          [self.dt, 0, 0],
                          [0, self.dt, 0],
                          [0, 0, self.dt]])
        
    def step(self, state):
        # compute acceleration vector from gravity assuming points mass of the earth and satelite
        if not self.elapsed_time == self.time:
            # [x,y,z]/|[x,y,z]| <- assumes point mass earth with center at (0,0,0)
            r_hat = state[:3]/np.linalg.norm(state[:3])
#             print(r_hat)
            a = -physics.Consts.G * qubesat.mass * planet.mass / np.linalg.norm(state[:3])**2 * r_hat
            new_state = np.dot(self.A, state) + np.dot(self.B, a)
            self.elapsed_time += self.dt
            return (self.elapsed_time, new_state)
        else:
            return (self.elapsed_time, None)

In [27]:
%matplotlib widget
import matplotlib.pyplot as plt
simulator = Simulator(10000, cubesat, planet, 0.1)
state = np.array([physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude), 0])

# while state is not None:
#     state_x.append(state[0])
#     state_y.append(state[1])
#     state_z.append(state[2])
#     time, state = simulator.step(state)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)
ax.plot(state_x, state_y)

# 2d plot
state_x = []
state_y = []
state_z = []
plotter, = ax.plot([])

def update(state_x):
    state = simulator.step(state)
    
    state_x.append(state[0])
#     state_y.append(state[1])
#     state_z.append(state[2])
    
    plotter.set_data(np.arange(0, len(state_x), 1), state_x)
    ax.set_xlim(0, len(state_x))

import matplotlib.animation as animation
line_ani = animation.FuncAnimation(fig, update, 20, fargs=(state_x),
                                   interval=1, blit=False)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
points = np.array([[-1, -1, -1],
                   [1, -1, -1 ],
                   [1, 1, -1],
                   [-1, 1, -1],
                   [-1, -1, 1],
                   [1, -1, 1 ],
                   [1, 1, 1],
                   [-1, 1, 1]])

def points_to_verts(Z):
    return [[Z[0],Z[1],Z[2],Z[3]],
            [Z[4],Z[5],Z[6],Z[7]], 
            [Z[0],Z[1],Z[5],Z[4]], 
            [Z[2],Z[3],Z[7],Z[6]], 
            [Z[1],Z[2],Z[6],Z[5]],
            [Z[4],Z[7],Z[3],Z[0]]]

def build_rot_mat(theta):
    return np.array([[np.sin(theta), np.cos(theta), 0],
                  [np.cos(theta), -np.sin(theta), 0],
                  [0, 0, 1]])

def rotate_points(points, theta):
    return np.array([np.dot(build_rot_mat(theta), point) for point in points])

# Attaching 3D axis to the figure
fig = plt.figure(figsize=(15,5))
ax  = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

# 3d simulation
collection = Poly3DCollection(points_to_verts(points), facecolors='grey', linewidths=1, edgecolors='black', alpha=0.9)
ax.add_collection3d(collection)

# 2d plot
line = []
plotter, = ax2.plot([], [])

def update(num, points, line):
    points = rotate_points(points, 0.01*num)
    ax.collections[0].set_verts(points_to_verts(points))
    line.append([0.01*num]) 
    plotter.set_data(np.arange(0, len(line), 1), line)
    ax2.set_xlim(0, len(line))


# Setting the axes properties
ax.set_xlim3d([-2.0, 2.0])
ax.set_xlabel('X')
ax.set_ylim3d([-2.0, 2.0])
ax.set_ylabel('Y')
ax.set_zlim3d([-2.0, 2.0])
ax.set_zlabel('Z')
ax.set_title('3D Simulation')

ax2.set_ylim(0, 0.5)

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update, 20, fargs=(points, line),
                                   interval=100, blit=False)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'Poly3DCollection' is not defined

# References
[Passive Magentic Attitude Control fro Cubesat Spacecraft](https://lasp.colorado.edu/home/csswe/files/2012/06/Gerhardt_SSC10_PMAC.pdf)

[Comparison of Control Laws for Magnetic Detumbling](https://www.researchgate.net/publication/263008407_Comparison_of_Control_Laws_for_Magnetic_Detumbling)
[]()


https://docs.google.com/document/d/16QsLCta1YLF__5pwcChfcjYQvqa9x6MZi19Ohj2Yuls/edit

https://www.youtube.com/playlist?list=PL_D7_GvGz-v3mDQ9iR-cfjXsQf4DeR1_H

https://pypi.org/project/pyIGRF/

https://stackoverflow.com/questions/53074908/map-an-image-onto-a-sphere-and-plot-3d-trajectories


In [7]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def mpl_sphere(image_file):
    img = plt.imread(image_file)

    # define a grid matching the map size, subsample along with pixels
    theta = np.linspace(0, np.pi, img.shape[0])
    phi = np.linspace(0, 2*np.pi, img.shape[1])

    count = 100 # keep 180 points along theta and phi
    theta_inds = np.linspace(0, img.shape[0] - 1, count).round().astype(int)
    phi_inds = np.linspace(0, img.shape[1] - 1, count).round().astype(int)
    theta = theta[theta_inds]
    phi = phi[phi_inds]
    img = img[np.ix_(theta_inds, phi_inds)]

    theta,phi = np.meshgrid(theta, phi)
    R = 1

    # sphere
    x = R * np.sin(theta) * np.cos(phi)
    y = R * np.sin(theta) * np.sin(phi)
    z = R * np.cos(theta)

    # create 3d Axes
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='auto', projection='3d')
    ax.plot_surface(x.T, y.T, z.T, facecolors=img/255, cstride=1, rstride=1) # we've already pruned ourselves

    # make the plot more spherical
    ax.set_aspect('equal')
    plt.show()

In [3]:
from mayavi import mlab
mlab.init_notebook()
from tvtk.api import tvtk

def auto_sphere(image_file):
    # create a figure window (and scene)
    fig = mlab.figure(size=(600, 600))

    # load and map the texture
    img = tvtk.JPEGReader()
    img.file_name = image_file
    texture = tvtk.Texture(input_connection=img.output_port, interpolate=1)
    # (interpolate for a less raster appearance when zoomed in)

    # use a TexturedSphereSource, a.k.a. getting our hands dirty
    R = 1
    Nrad = 180

    # create the sphere source with a given radius and angular resolution
    sphere = tvtk.TexturedSphereSource(radius=R, theta_resolution=Nrad,
                                       phi_resolution=Nrad)

    # assemble rest of the pipeline, assign texture    
    sphere_mapper = tvtk.PolyDataMapper(input_connection=sphere.output_port)
    sphere_actor = tvtk.Actor(mapper=sphere_mapper, texture=texture)
    fig.scene.add_actor(sphere_actor)


image_file = 'earth.jpg'
auto_sphere(image_file)
mlab.show()

Notebook initialized with ipy backend.
